**AP: is this a temporary or work in progress notebook? If yes, maybe find another name**

After reading a zarr file, I re-chunk the data using xarray.Dataset.chunk. 
Then create a newly chunked data stored as zarr file with xarray.Dataset.to_zarr
But I get error message: 'notimplemented error 
my xarray version is12.1, 
This looks like the bug which is already fixed at https://github.com/pydata/xarray/issues/2300 ...Why do I get 'notimplemented error ?
Do I have to use 'del dsread.data.encoding['chunks']. each time before using 'Dataset.to_zarr' as a workaround? or may be I am missing somthing.  I hope someone can help me..

# 0. setting up enviroments,  create a xarray.Dataset and save as zarr file. 

In [1]:
import dask
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
print(xr.__version__)

0.12.1


In [3]:
data = np.random.rand(200,100,10000)
#Dimensions  = ['lat','lon','time']
times = pd.date_range('2001-01-01',periods=10000,freq='H')
lon= np.linspace(0,49,100)
lat = np.linspace(0,49,200)
#lat = np.arange(-5,5,0.1)
da = xr.DataArray(data, coords=[lat,lon,times], dims=['lat','lon','time'])
ds = xr.Dataset({'data':da})
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 100, time: 10000)
Coordinates:
  * lat      (lat) float64 0.0 0.2462 0.4925 0.7387 ... 48.26 48.51 48.75 49.0
  * lon      (lon) float64 0.0 0.4949 0.9899 1.485 ... 47.52 48.01 48.51 49.0
  * time     (time) datetime64[ns] 2001-01-01 ... 2002-02-21T15:00:00
Data variables:
    data     (lat, lon, time) float64 0.1008 0.5696 0.4781 ... 0.4254 0.3909

 data size: 1.6 GB


In [4]:
ds =  ds.chunk({'lat': -1, 'lon': -1, 'time': 1})
xr.DataArray(data)
ds

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 100, time: 10000)
Coordinates:
  * lat      (lat) float64 0.0 0.2462 0.4925 0.7387 ... 48.26 48.51 48.75 49.0
  * lon      (lon) float64 0.0 0.4949 0.9899 1.485 ... 47.52 48.01 48.51 49.0
  * time     (time) datetime64[ns] 2001-01-01 ... 2002-02-21T15:00:00
Data variables:
    data     (lat, lon, time) float64 dask.array<shape=(200, 100, 10000), chunksize=(200, 100, 1)>

In [5]:
%time ds.to_zarr('ds.zarr',  mode='w')

CPU times: user 11.5 s, sys: 5.74 s, total: 17.2 s
Wall time: 8.62 s


# 1.read the zarr file and re-chunk

In [6]:
dask.config.get('array.chunk-size')

'128MiB'

In [7]:
dstmp =xr.open_zarr('./ds.zarr')
dsread =  dstmp.chunk({'lat': 'auto', 'lon': 'auto', 'time': -1})
print(dsread)
print('\n data size: %.1f GB' %(dsread.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 100, time: 10000)
Coordinates:
  * lat      (lat) float64 0.0 0.2462 0.4925 0.7387 ... 48.26 48.51 48.75 49.0
  * lon      (lon) float64 0.0 0.4949 0.9899 1.485 ... 47.52 48.01 48.51 49.0
  * time     (time) datetime64[ns] 2001-01-01 ... 2002-02-21T15:00:00
Data variables:
    data     (lat, lon, time) float64 dask.array<shape=(200, 100, 10000), chunksize=(50, 25, 10000)>

 data size: 1.6 GB


# 2. write the re-chunked data to a file, then err message as following.
'NotImplementedError: Specified zarr chunks (200, 100, 1) would overlap multiple dask chunks ((50, 50, 50, 50), (25, 25, 25, 25), (10000,)). This is not implemented in xarray yet.  Consider rechunking the data using `chunk()` or specifying different chunks in encoding.'

In [8]:
%time dsread.to_zarr('./dsread.zarr',  mode='w')

NotImplementedError: Specified zarr chunks (200, 100, 1) would overlap multiple dask chunks ((50, 50, 50, 50), (25, 25, 25, 25), (10000,)). This is not implemented in xarray yet.  Consider rechunking the data using `chunk()` or specifying different chunks in encoding.

In [9]:
print(dsread.data.encoding)
print(dsread)

{'chunks': (200, 100, 1), 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, '_FillValue': nan, 'dtype': dtype('float64')}
<xarray.Dataset>
Dimensions:  (lat: 200, lon: 100, time: 10000)
Coordinates:
  * lat      (lat) float64 0.0 0.2462 0.4925 0.7387 ... 48.26 48.51 48.75 49.0
  * lon      (lon) float64 0.0 0.4949 0.9899 1.485 ... 47.52 48.01 48.51 49.0
  * time     (time) datetime64[ns] 2001-01-01 ... 2002-02-21T15:00:00
Data variables:
    data     (lat, lon, time) float64 dask.array<shape=(200, 100, 10000), chunksize=(50, 25, 10000)>


# 3. Workaround: delete encoding before saving the xarray.Dataset as zarr. Do we have to do this each time? why?

In [10]:
del dsread.data.encoding['chunks']

In [11]:
%time dsread.to_zarr('./dsread.zarr',  mode='w')

CPU times: user 11.5 s, sys: 10.4 s, total: 21.9 s
Wall time: 12.2 s


In [12]:
dsnew =xr.open_zarr('./dsread.zarr')
print(dsnew)
print(dsnew.data.encoding)

<xarray.Dataset>
Dimensions:  (lat: 200, lon: 100, time: 10000)
Coordinates:
  * lat      (lat) float64 0.0 0.2462 0.4925 0.7387 ... 48.26 48.51 48.75 49.0
  * lon      (lon) float64 0.0 0.4949 0.9899 1.485 ... 47.52 48.01 48.51 49.0
  * time     (time) datetime64[ns] 2001-01-01 ... 2002-02-21T15:00:00
Data variables:
    data     (lat, lon, time) float64 dask.array<shape=(200, 100, 10000), chunksize=(50, 25, 10000)>
{'chunks': (50, 25, 10000), 'compressor': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0), 'filters': None, '_FillValue': nan, 'dtype': dtype('float64')}
